<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/03_xyz_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-27 01:57:20--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12110 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.83K  --.-KB/s    in 0s      

2022-06-27 01:57:20 (54.1 MB/s) - ‘utils.py’ saved [12110/12110]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import ModelCreator, get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 1.3 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3510 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
xyz_creator = ModelCreator(HUB_URL, 'XYZ-model')
xyz_model = xyz_creator.make_model(img_size=IMG_SIZE)

Model: "XYZ-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              12930622  
                                                                 
 dense (Dense)               (None, 16)                24592     
                                                                 
Total params: 12,955,214
Trainable params: 12,845,998
Non-trainable params: 109,216
_________________________________________________________________


In [10]:
callbacks = xyz_creator.get_callbacks()

In [11]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=15,
                        class_weight=class_weight,
                        validation_data=val_ds)

Epoch 1/15
519/519 [==============================] - 3222s 6s/step - loss: 16.2919 - auc: 0.7512 - accuracy: 0.6674 - val_loss: 0.9552 - val_auc: 0.8143 - val_accuracy: 0.7274
Epoch 2/15
519/519 [==============================] - 253s 484ms/step - loss: 10.1347 - auc: 0.8770 - accuracy: 0.7920 - val_loss: 0.7196 - val_auc: 0.8677 - val_accuracy: 0.7769
Epoch 3/15
519/519 [==============================] - 252s 482ms/step - loss: 7.4890 - auc: 0.9193 - accuracy: 0.8387 - val_loss: 1.1594 - val_auc: 0.6972 - val_accuracy: 0.6416
Epoch 4/15
519/519 [==============================] - 248s 476ms/step - loss: 5.9527 - auc: 0.9381 - accuracy: 0.8644 - val_loss: 0.8191 - val_auc: 0.8572 - val_accuracy: 0.7838
Epoch 5/15
519/519 [==============================] - 247s 474ms/step - loss: 4.2674 - auc: 0.9626 - accuracy: 0.9040 - val_loss: 1.0428 - val_auc: 0.8202 - val_accuracy: 0.7516
Epoch 6/15
519/519 [==============================] - 250s 479ms/step - loss: 3.3987 - auc: 0.9709 - accuracy:

In [12]:
history = xyz_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=20,
                          initial_epoch=15,
                          validation_data=val_ds,
                          class_weight=class_weight)

Epoch 16/20
519/519 [==============================] - 248s 475ms/step - loss: 0.2958 - auc: 0.9989 - accuracy: 0.9915 - val_loss: 1.2497 - val_auc: 0.8363 - val_accuracy: 0.8214
Epoch 17/20
519/519 [==============================] - 245s 470ms/step - loss: 0.3538 - auc: 0.9984 - accuracy: 0.9909 - val_loss: 1.2278 - val_auc: 0.8346 - val_accuracy: 0.8199
Epoch 18/20
519/519 [==============================] - 247s 473ms/step - loss: 0.2496 - auc: 0.9987 - accuracy: 0.9942 - val_loss: 1.2720 - val_auc: 0.8324 - val_accuracy: 0.8157
Epoch 19/20
519/519 [==============================] - 248s 475ms/step - loss: 0.1701 - auc: 0.9992 - accuracy: 0.9955 - val_loss: 1.2600 - val_auc: 0.8380 - val_accuracy: 0.8248
Epoch 20/20
519/519 [==============================] - 252s 483ms/step - loss: 0.2340 - auc: 0.9987 - accuracy: 0.9932 - val_loss: 1.1638 - val_auc: 0.8420 - val_accuracy: 0.8254


In [13]:
!cp ./logs/checkpoints/XYZ-model.h5 ./drive/MyDrive/checkpoints/xyz-model.h5